# Cryptocurrency Data Processing Pipeline
## Processing Combined Multi-Crypto Dataset

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.8f' % x)

print("✅ Libraries imported successfully!")
print(f"Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Libraries imported successfully!
Current date: 2025-11-14 17:34:50


## 1. Load Combined Dataset

In [2]:
# Load the combined cryptocurrency data
df = pd.read_csv('crypto_data_combined.csv')

print("=" * 70)
print("DATASET LOADED")
print("=" * 70)
print(f"Dataset shape: {df.shape}")
print(f"Total records: {len(df):,}")
print(f"Unique cryptocurrencies: {df['Symbol'].nunique()}")
print(f"\nColumn names: {df.columns.tolist()}")
print("=" * 70)

print("\nFirst few rows:")
df.head(10)

DATASET LOADED
Dataset shape: (52287, 130)
Total records: 52,287
Unique cryptocurrencies: 25

Column names: ['SNo', 'Name', 'Symbol', 'Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Marketcap', 'High.1', 'Low.1', 'Open.1', 'Close.1', 'Volume.1', 'High.2', 'Low.2', 'Open.2', 'Close.2', 'Volume.2', 'High.3', 'Low.3', 'Open.3', 'Close.3', 'Volume.3', 'High.4', 'Low.4', 'Open.4', 'Close.4', 'Volume.4', 'High.5', 'Low.5', 'Open.5', 'Close.5', 'Volume.5', 'High.6', 'Low.6', 'Open.6', 'Close.6', 'Volume.6', 'High.7', 'Low.7', 'Open.7', 'Close.7', 'Volume.7', 'High.8', 'Low.8', 'Open.8', 'Close.8', 'Volume.8', 'High.9', 'Low.9', 'Open.9', 'Close.9', 'Volume.9', 'High.10', 'Low.10', 'Open.10', 'Close.10', 'Volume.10', 'High.11', 'Low.11', 'Open.11', 'Close.11', 'Volume.11', 'High.12', 'Low.12', 'Open.12', 'Close.12', 'Volume.12', 'High.13', 'Low.13', 'Open.13', 'Close.13', 'Volume.13', 'High.14', 'Low.14', 'Open.14', 'Close.14', 'Volume.14', 'High.15', 'Low.15', 'Open.15', 'Close.15', 'Volum

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,High.1,Low.1,Open.1,Close.1,Volume.1,High.2,Low.2,Open.2,Close.2,Volume.2,High.3,Low.3,Open.3,Close.3,Volume.3,High.4,Low.4,Open.4,Close.4,Volume.4,High.5,Low.5,Open.5,Close.5,Volume.5,High.6,Low.6,Open.6,Close.6,Volume.6,High.7,Low.7,Open.7,Close.7,Volume.7,High.8,Low.8,Open.8,Close.8,Volume.8,High.9,Low.9,Open.9,Close.9,Volume.9,High.10,Low.10,Open.10,Close.10,Volume.10,High.11,Low.11,Open.11,Close.11,Volume.11,High.12,Low.12,Open.12,Close.12,Volume.12,High.13,Low.13,Open.13,Close.13,Volume.13,High.14,Low.14,Open.14,Close.14,Volume.14,High.15,Low.15,Open.15,Close.15,Volume.15,High.16,Low.16,Open.16,Close.16,Volume.16,High.17,Low.17,Open.17,Close.17,Volume.17,High.18,Low.18,Open.18,Close.18,Volume.18,High.19,Low.19,Open.19,Close.19,Volume.19,High.20,Low.20,Open.20,Close.20,Volume.20,High.21,Low.21,Open.21,Close.21,Volume.21,High.22,Low.22,Open.22,Close.22,Volume.22,High.23,Low.23,Open.23,Close.23,Volume.23,High.24,Low.24,Open.24,Close.24,Volume.24
0,NaN,NaN,NaN,NaN,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,NaN,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ADA-USD,ADA-USD,ADA-USD,ADA-USD,ADA-USD,BNB-USD,BNB-USD,BNB-USD,BNB-USD,BNB-USD,XRP-USD,XRP-USD,XRP-USD,XRP-USD,XRP-USD,SOL-USD,SOL-USD,SOL-USD,SOL-USD,SOL-USD,DOT-USD,DOT-USD,DOT-USD,DOT-USD,DOT-USD,DOGE-USD,DOGE-USD,DOGE-USD,DOGE-USD,DOGE-USD,MATIC-USD,MATIC-USD,MATIC-USD,MATIC-USD,MATIC-USD,AVAX-USD,AVAX-USD,AVAX-USD,AVAX-USD,AVAX-USD,LINK-USD,LINK-USD,LINK-USD,LINK-USD,LINK-USD,UNI-USD,UNI-USD,UNI-USD,UNI-USD,UNI-USD,LTC-USD,LTC-USD,LTC-USD,LTC-USD,LTC-USD,ATOM-USD,ATOM-USD,ATOM-USD,ATOM-USD,ATOM-USD,XLM-USD,XLM-USD,XLM-USD,XLM-USD,XLM-USD,ALGO-USD,ALGO-USD,ALGO-USD,ALGO-USD,ALGO-USD,VET-USD,VET-USD,VET-USD,VET-USD,VET-USD,TRX-USD,TRX-USD,TRX-USD,TRX-USD,TRX-USD,FIL-USD,FIL-USD,FIL-USD,FIL-USD,FIL-USD,ETC-USD,ETC-USD,ETC-USD,ETC-USD,ETC-USD,XMR-USD,XMR-USD,XMR-USD,XMR-USD,XMR-USD,AAVE-USD,AAVE-USD,AAVE-USD,AAVE-USD,AAVE-USD,THETA-USD,THETA-USD,THETA-USD,THETA-USD,THETA-USD,EOS-USD,EOS-USD,EOS-USD,EOS-USD,EOS-USD,XTZ-USD,XTZ-USD,XTZ-USD,XTZ-USD,XTZ-USD
1,1.00000000,Aave,AAVE,2020-10-02,NaN,NaN,NaN,NaN,NaN,0.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5165709853172302,0.0,0.0,0.5165709853172302,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.00000000,Aave,AAVE,2020-10-03,NaN,NaN,NaN,NaN,NaN,0.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.30593872070312,0.5238010287284851,0.5238360166549683,53.1514892578125,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.00000000,Aave,AAVE,2020-10-04,NaN,NaN,NaN,NaN,NaN,0.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.07035446166992,50.68899154663086,53.17985916137695,52.67503356933594,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.00000000,Aave,AAVE,2020-10-05,NaN,NaN,NaN,NaN,NaN,0.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [3]:
# Data info and missing values
print("\nDataset Information:")
print(df.info())

print("\n" + "=" * 70)
print("MISSING VALUES CHECK")
print("=" * 70)
missing = df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("✅ No missing values found!")

print("\n" + "=" * 70)
print("CRYPTOCURRENCY DISTRIBUTION")
print("=" * 70)
print(df['Symbol'].value_counts())


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52287 entries, 0 to 52286
Columns: 130 entries, SNo to Volume.24
dtypes: float64(2), object(128)
memory usage: 51.9+ MB
None

MISSING VALUES CHECK
SNo              1
Name             1
Symbol           1
Date             1
High         50142
             ...  
High.24      50142
Low.24       50142
Open.24      50142
Close.24     50142
Volume.24    50142
Length: 130, dtype: int64

CRYPTOCURRENCY DISTRIBUTION
Symbol
ADA      2144
DOGE     2144
ALGO     2144
ATOM     2144
BNB      2144
BTC      2144
ETH      2144
ETC      2144
EOS      2144
XRP      2144
XMR      2144
VET      2144
FIL      2144
LINK     2144
LTC      2144
TRX      2144
THETA    2144
XTZ      2144
XLM      2144
SOL      2044
UNI      1934
DOT      1912
MATIC    1910
AVAX     1881
AAVE     1869
Name: count, dtype: int64


## 2. Data Cleaning

In [5]:
# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Handle missing values (if any)
initial_rows = len(df)
df = df.dropna(subset=['Close', 'Volume', 'Open', 'High', 'Low'])
dropped_rows = initial_rows - len(df)

# Remove duplicates
initial_rows = len(df)
df = df.drop_duplicates(subset=['Symbol', 'Date'])
duplicate_rows = initial_rows - len(df)

# Sort by Symbol and Date
df = df.sort_values(['Symbol', 'Date']).reset_index(drop=True)
# Convert price columns to numeric
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
df['High'] = pd.to_numeric(df['High'], errors='coerce')
df['Low'] = pd.to_numeric(df['Low'], errors='coerce')

# Then filter
df = df[(df['Close'] > 0) & (df['Open'] > 0) & (df['High'] > 0) & (df['Low'] > 0)]

# Remove any rows with zero or negative prices
df = df[(df['Close'] > 0) & (df['Open'] > 0) & (df['High'] > 0) & (df['Low'] > 0)]

print("=" * 70)
print("DATA CLEANING SUMMARY")
print("=" * 70)
print(f"Rows dropped (missing values): {dropped_rows}")
print(f"Duplicate rows removed: {duplicate_rows}")
print(f"Cleaned dataset shape: {df.shape}")
print(f"Date range: {df['Date'].min().date()} to {df['Date'].max().date()}")
print(f"Total trading days: {df['Date'].nunique()}")
print(f"Cryptocurrencies: {df['Symbol'].nunique()}")
print("=" * 70)

DATA CLEANING SUMMARY
Rows dropped (missing values): 0
Duplicate rows removed: 0
Cleaned dataset shape: (2144, 130)
Date range: 2020-01-01 to 2025-11-13
Total trading days: 2144
Cryptocurrencies: 1


## 3. Feature Engineering - Technical Indicators

In [6]:
# Create technical indicators and features
def add_technical_indicators(df):
    """
    Add technical indicators to the dataframe
    """
    df_copy = df.copy()
    
    # Price features
    df_copy['Price_Change'] = df_copy['Close'] - df_copy['Open']
    df_copy['Price_Change_Pct'] = (df_copy['Price_Change'] / df_copy['Open']) * 100
    df_copy['Daily_Range'] = df_copy['High'] - df_copy['Low']
    df_copy['Volatility'] = (df_copy['Daily_Range'] / df_copy['High']) * 100
    
    # Average price
    df_copy['Avg_Price'] = (df_copy['High'] + df_copy['Low'] + df_copy['Close']) / 3
    
    # Typical Price (HLC/3)
    df_copy['Typical_Price'] = (df_copy['High'] + df_copy['Low'] + df_copy['Close']) / 3
    
    # Body size (candle)
    df_copy['Body_Size'] = abs(df_copy['Close'] - df_copy['Open'])
    df_copy['Body_Size_Pct'] = (df_copy['Body_Size'] / df_copy['Open']) * 100
    
    # Upper and Lower shadows
    df_copy['Upper_Shadow'] = df_copy['High'] - df_copy[['Open', 'Close']].max(axis=1)
    df_copy['Lower_Shadow'] = df_copy[['Open', 'Close']].min(axis=1) - df_copy['Low']
    
    # Bullish/Bearish indicator
    df_copy['Is_Bullish'] = (df_copy['Close'] > df_copy['Open']).astype(int)
    
    return df_copy

print("Adding technical indicators...")
df_processed = add_technical_indicators(df)
print(f"✅ Technical indicators added! New shape: {df_processed.shape}")
print(f"New columns: {df_processed.shape[1] - df.shape[1]}")

df_processed.head()

Adding technical indicators...
✅ Technical indicators added! New shape: (2144, 141)
New columns: 11


,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,High.1,Low.1,Open.1,Close.1,Volume.1,High.2,Low.2,Open.2,Close.2,Volume.2,High.3,Low.3,Open.3,Close.3,Volume.3,High.4,Low.4,Open.4,Close.4,Volume.4,High.5,Low.5,Open.5,Close.5,Volume.5,High.6,Low.6,Open.6,Close.6,Volume.6,High.7,Low.7,Open.7,Close.7,Volume.7,High.8,Low.8,Open.8,Close.8,Volume.8,High.9,Low.9,Open.9,Close.9,Volume.9,High.10,Low.10,Open.10,Close.10,Volume.10,High.11,Low.11,Open.11,Close.11,Volume.11,High.12,Low.12,Open.12,Close.12,Volume.12,High.13,Low.13,Open.13,Close.13,Volume.13,High.14,Low.14,Open.14,Close.14,Volume.14,High.15,Low.15,Open.15,Close.15,Volume.15,High.16,Low.16,Open.16,Close.16,Volume.16,High.17,Low.17,Open.17,Close.17,Volume.17,High.18,Low.18,Open.18,Close.18,Volume.18,High.19,Low.19,Open.19,Close.19,Volume.19,High.20,Low.20,Open.20,Close.20,Volume.20,High.21,Low.21,Open.21,Close.21,Volume.21,High.22,Low.22,Open.22,Close.22,Volume.22,High.23,Low.23,Open.23,Close.23,Volume.23,High.24,Low.24,Open.24,Close.24,Volume.24,Price_Change,Price_Change_Pct,Daily_Range,Volatility,Avg_Price,Typical_Price,Body_Size,Body_Size_Pct,Upper_Shadow,Lower_Shadow,Is_Bullish
0,12327.00000000,Bitcoin,BTC,2020-01-01,7254.33056641,7174.94433594,7194.89208984,7200.17431641,18565664997.00000000,133676024278401.92187500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.28222656,0.07341634,79.38623047,1.09432883,7209.81640625,7209.81640625,5.28222656,0.07341634,54.15625000,19.94775391,1
1,12328.00000000,Bitcoin,BTC,2020-01-02,7212.15527344,6935.27001953,7202.55126953,6985.47021484,20802083465.00000000,145312334451451.15625000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-217.08105469,-3.01394668,276.88525391,3.83914715,7044.29850260,7044.29850260,217.08105469,3.01394668,9.60400391,50.20019531,0
2,12329.00000000,Bitcoin,BTC,2020-01-03,7413.71533203,6914.99609375,6984.42871094,7344.88427734,28111481032.00000000,206475575044783.84375000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.45556641,5.16084538,498.71923828,6.72698122,7224.53190104,7224.53190104,360.45556641,5.16084538,68.83105469,69.43261719,1
3,12330.00000000,Bitcoin,BTC,2020-01-04,7427.38574219,7309.51416016,7345.37548828,7410.65673828,18444271275.00000000,136684163206766.04687500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

## 4. Feature Engineering - Moving Averages

In [7]:
# Add moving averages and rolling statistics
def add_rolling_features(df, windows=[7, 14, 21, 30]):
    """
    Add moving averages and rolling statistics
    Calculated per cryptocurrency (grouped by Symbol)
    """
    df_copy = df.copy()
    
    print(f"Calculating rolling features for windows: {windows}")
    
    for window in windows:
        print(f"  - Processing {window}-day window...")
        
        # Moving averages (SMA)
        df_copy[f'SMA_{window}'] = df_copy.groupby('Symbol')['Close'].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        
        # Exponential Moving Average (EMA)
        df_copy[f'EMA_{window}'] = df_copy.groupby('Symbol')['Close'].transform(
            lambda x: x.ewm(span=window, adjust=False).mean()
        )
        
        # Rolling volatility (standard deviation)
        df_copy[f'Rolling_Std_{window}'] = df_copy.groupby('Symbol')['Close'].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )
        
        # Rolling volume average
        df_copy[f'Volume_MA_{window}'] = df_copy.groupby('Symbol')['Volume'].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        
        # Rolling min and max
        df_copy[f'Rolling_Max_{window}'] = df_copy.groupby('Symbol')['High'].transform(
            lambda x: x.rolling(window=window, min_periods=1).max()
        )
        df_copy[f'Rolling_Min_{window}'] = df_copy.groupby('Symbol')['Low'].transform(
            lambda x: x.rolling(window=window, min_periods=1).min()
        )
    
    return df_copy

df_processed = add_rolling_features(df_processed)
print(f"\n✅ Rolling features added! New shape: {df_processed.shape}")

Calculating rolling features for windows: [7, 14, 21, 30]
  - Processing 7-day window...
  - Processing 14-day window...
  - Processing 21-day window...
  - Processing 30-day window...

✅ Rolling features added! New shape: (2144, 165)


## 5. Feature Engineering - Lag Features

In [8]:
# Add lag features (previous day values)
def add_lag_features(df, lags=[1, 2, 3, 5, 7]):
    """
    Add lag features (previous values)
    Grouped by Symbol to avoid mixing different cryptocurrencies
    """
    df_copy = df.copy()
    
    print(f"Adding lag features for periods: {lags}")
    
    for lag in lags:
        print(f"  - Creating {lag}-day lag features...")
        
        # Lag close prices
        df_copy[f'Close_Lag_{lag}'] = df_copy.groupby('Symbol')['Close'].shift(lag)
        
        # Lag volume
        df_copy[f'Volume_Lag_{lag}'] = df_copy.groupby('Symbol')['Volume'].shift(lag)
        
        # Lag returns
        df_copy[f'Return_Lag_{lag}'] = df_copy.groupby('Symbol')['Price_Change_Pct'].shift(lag)
        
        # Lag volatility
        df_copy[f'Volatility_Lag_{lag}'] = df_copy.groupby('Symbol')['Volatility'].shift(lag)
    
    return df_copy

df_processed = add_lag_features(df_processed)
print(f"\n✅ Lag features added! New shape: {df_processed.shape}")

Adding lag features for periods: [1, 2, 3, 5, 7]
  - Creating 1-day lag features...
  - Creating 2-day lag features...
  - Creating 3-day lag features...
  - Creating 5-day lag features...
  - Creating 7-day lag features...

✅ Lag features added! New shape: (2144, 185)


## 6. Feature Engineering - Time-Based Features

In [9]:
# Add time-based features
print("Adding time-based features...")

df_processed['Year'] = df_processed['Date'].dt.year
df_processed['Month'] = df_processed['Date'].dt.month
df_processed['Day'] = df_processed['Date'].dt.day
df_processed['DayOfWeek'] = df_processed['Date'].dt.dayofweek  # Monday=0, Sunday=6
df_processed['Quarter'] = df_processed['Date'].dt.quarter
df_processed['DayOfYear'] = df_processed['Date'].dt.dayofyear
df_processed['WeekOfYear'] = df_processed['Date'].dt.isocalendar().week

# Is weekend?
df_processed['Is_Weekend'] = (df_processed['DayOfWeek'] >= 5).astype(int)

# Month start/end
df_processed['Is_Month_Start'] = df_processed['Date'].dt.is_month_start.astype(int)
df_processed['Is_Month_End'] = df_processed['Date'].dt.is_month_end.astype(int)

print(f"✅ Time-based features added! Final shape: {df_processed.shape}")

Adding time-based features...
✅ Time-based features added! Final shape: (2144, 195)


## 7. Target Variable - Next Day Close Price

In [10]:
# Create target variable: Next day's closing price
print("Creating target variable (Next Day Close Price)...")

df_processed['Next_Day_Close'] = df_processed.groupby('Symbol')['Close'].shift(-1)

# Also create target for price change
df_processed['Next_Day_Price_Change'] = df_processed['Next_Day_Close'] - df_processed['Close']
df_processed['Next_Day_Price_Change_Pct'] = (
    (df_processed['Next_Day_Close'] - df_processed['Close']) / df_processed['Close']
) * 100

# Binary classification target (Up/Down)
df_processed['Next_Day_Direction'] = (df_processed['Next_Day_Close'] > df_processed['Close']).astype(int)

print("✅ Target variables created!")
print("\nTarget variables:")
print("  - Next_Day_Close: Actual next day closing price (Regression)")
print("  - Next_Day_Price_Change_Pct: Percentage change (Regression)")
print("  - Next_Day_Direction: 1=Up, 0=Down (Classification)")

Creating target variable (Next Day Close Price)...
✅ Target variables created!

Target variables:
  - Next_Day_Close: Actual next day closing price (Regression)
  - Next_Day_Price_Change_Pct: Percentage change (Regression)
  - Next_Day_Direction: 1=Up, 0=Down (Classification)


## 8. Final Data Quality Check

In [11]:
print("=" * 70)
print("FINAL PROCESSED DATASET SUMMARY")
print("=" * 70)
print(f"Total records: {len(df_processed):,}")
print(f"Total features: {df_processed.shape[1]}")
print(f"Cryptocurrencies: {df_processed['Symbol'].nunique()}")
print(f"Date range: {df_processed['Date'].min().date()} to {df_processed['Date'].max().date()}")

print("\nMissing values per column:")
missing = df_processed.isnull().sum()
missing_pct = (missing / len(df_processed)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing[missing > 0],
    'Missing_Pct': missing_pct[missing > 0]
}).sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
else:
    print("✅ No missing values!")

print("\nFeature categories:")
feature_cols = [col for col in df_processed.columns if col not in ['SNo', 'Name', 'Symbol', 'Date']]
print(f"  - Total features: {len(feature_cols)}")
print("=" * 70)

FINAL PROCESSED DATASET SUMMARY
Total records: 2,144
Total features: 199
Cryptocurrencies: 1
Date range: 2020-01-01 to 2025-11-13

Missing values per column:
                           Missing_Count  Missing_Pct
High.1                              2144 100.00000000
Low.1                               2144 100.00000000
Open.1                              2144 100.00000000
Close.1                             2144 100.00000000
Volume.1                            2144 100.00000000
...                                  ...          ...
Return_Lag_1                           1   0.04664179
Rolling_Std_30                         1   0.04664179
Next_Day_Close                         1   0.04664179
Next_Day_Price_Change                  1   0.04664179
Next_Day_Price_Change_Pct              1   0.04664179

[147 rows x 2 columns]

Feature categories:
  - Total features: 195


## 9. Save Processed Data

In [12]:
# Save the processed dataset
output_file = 'crypto_data_processed.csv'
df_processed.to_csv(output_file, index=False)
print(f"✅ Processed data saved to: {output_file}")

# Save statistics per coin
print("\nGenerating coin-wise statistics...")
coin_stats = df_processed.groupby('Symbol').agg({
    'Name': 'first',
    'Close': ['count', 'mean', 'std', 'min', 'max'],
    'Volume': ['mean', 'sum'],
    'Volatility': 'mean',
    'Price_Change_Pct': ['mean', 'std'],
    'Date': ['min', 'max']
}).reset_index()

coin_stats.columns = ['_'.join(str(col)).strip('_') for col in coin_stats.columns.values]
coin_stats.to_csv('coin_statistics.csv', index=False)
print("✅ Coin statistics saved to: coin_statistics.csv")

# Save feature list
feature_list = pd.DataFrame({
    'Feature_Name': df_processed.columns.tolist(),
    'Data_Type': df_processed.dtypes.values.astype(str)
})
feature_list.to_csv('feature_list.csv', index=False)
print("✅ Feature list saved to: feature_list.csv")

print("\n" + "=" * 70)
print("DATA PROCESSING COMPLETE!")
print("=" * 70)
print("\nGenerated files:")
print("  1. crypto_data_processed.csv - Full processed dataset")
print("  2. coin_statistics.csv - Summary stats per cryptocurrency")
print("  3. feature_list.csv - List of all features")
print("\nNext step: Run 02_exploratory_data_analysis.ipynb")
print("=" * 70)

✅ Processed data saved to: crypto_data_processed.csv

Generating coin-wise statistics...
✅ Coin statistics saved to: coin_statistics.csv
✅ Feature list saved to: feature_list.csv

DATA PROCESSING COMPLETE!

Generated files:
  1. crypto_data_processed.csv - Full processed dataset
  2. coin_statistics.csv - Summary stats per cryptocurrency
  3. feature_list.csv - List of all features

Next step: Run 02_exploratory_data_analysis.ipynb


In [13]:
# Display sample of processed data
print("\nSample of processed data:")
df_processed[['Date', 'Symbol', 'Close', 'Next_Day_Close', 'Next_Day_Direction', 
              'SMA_7', 'SMA_14', 'Volatility', 'Price_Change_Pct']].head(15)


Sample of processed data:


,Date,Symbol,Close,Next_Day_Close,Next_Day_Direction,SMA_7,SMA_14,Volatility,Price_Change_Pct
0,2020-01-01,BTC,7200.17431641,6985.47021484,0,7200.17431641,7200.17431641,1.09432883,0.07341634
1,2020-01-02,BTC,6985.47021484,7344.88427734,1,7092.82226562,7092.82226562,3.83914715,-3.01394668
2,2020-01-03,BTC,7344.88427734,7410.65673828,1,7176.84293620,7176.84293620,6.72698122,5.16084538
3,2020-01-04,BTC,7410.65673828,7411.31738281,1,7235.29638672,7235.29638672,1.58698614,0.88873945
4,2020-01-05,BTC,7411.31738281,7769.21923828,1,7270.50058594,7270.50058594,1.90816465,0.01168245
5,2020-01-06,BTC,7769.21923828,8163.69238281,1,7353.62036133,7353.62036133,4.78772266,4.84136572
6,2020-01-07,BTC,8163.69238281,8079.86279297,0,7469.34493583,7469.34493583,5.01317513,5.08464946
7,2020-01-08,BTC,8079.86279297,7879.07128906,0,7595.01471819,7545.65966797,5.23969966,-1.00555504
8,2020-01-09,BTC,7879.07128906,8166.55419922,1,7722.67201451,7582.70540365,2.96811813,-2.51444159
9,2020-01-10,BTC,8166.55419922,8037.53759766,0,7840.05343192,7641.09028320,5.38512678,3.65873733
